In [1]:
import pandas as pd
import requests
import json
import sqlite3

# Define the API endpoint and access key
url = "https://api.weatherstack.com/current?access_key=696d3a695312dc11986589fce4d2a180"
querystring = {"query":"New Delhi"}

In [2]:
# Get user input
ans = False
while not ans:
    data = input('Would you like to explore olympics 2024 medals data or weather data? Choose olympics or weather: ').lower()
    # possible options are olympics or weather
    if data not in ['olympics', 'weather']:
        print("Please say either olympics or weather.")
    else:
        ans = True  # The input is valid, so we can exit the loop

print(f"You chose to explore {data}.")

Would you like to explore olympics 2024 medals data or weather data? Choose olympics or weather: olympics
You chose to explore olympics.


In [13]:
# Read in csv file
if (data == 'olympics'):
    df = pd.read_csv('olympics_2024_medals_table.csv')
    df.head(5)
    print("Data loaded successfully!")

    # Brief summary of the data after ingestion
    print(f"Data Ingestion Summary: {len(df)} records, {len(df.columns)} columns")

    # Rename columns for easier access
    df.rename(columns={'sliver':'silver', 'bronzs':'bronze'}, inplace=True)

    # Add column for gold %
    df['gold_pct'] = (df['gold'] / df['total']) * 100

    # Add column for silver %
    df['silver_pct'] = (df['silver'] / df['total']) * 100

    # Add column for bronze %
    df['bronze_pct'] = (df['bronze'] / df['total']) * 100

    # Brief summary of the data after ingestion
    print(f"Data Post Processing Summary: {len(df)} records, {len(df.columns)} columns")

elif (data == 'weather'):
    # Make the request
    response = requests.get(url, params=querystring)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the response JSON data
        data = response.json()

        # Save the data into a JSON file
        with open('current_weather.json', 'w') as json_file:
            json.dump(data, json_file, indent=4)

        print("Weather data saved to 'current_weather.json'")
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")

    df = pd.read_json('current_weather.json')

            # Brief summary of the data after ingestion
    print(f"Weather Data Ingestion Summary: {len(df)} records, {len(df.columns)} columns")

    # Drop 'requests' from the location data
    df.drop(columns=["request"], inplace=True)

    # Post processing: Generate summary after processing data
    print(f"Post Processing Summary: {len(df)} records, {len(df.columns)} columns")

Data loaded successfully!
Data Ingestion Summary: 91 records, 6 columns
Data Post Processing Summary: 91 records, 9 columns


In [9]:
# Get user input
ans = False
while not ans:
    data_form = input("What form do you want the data to be in? Choose either csv, json, or sql: ").lower()
    # possible options are sql, json, or csv
    if data_form not in ['csv', 'json', 'sql']:
        print("Please say either csv, json, or sql.")
    else:
        ans = True  # The input is valid, so we can exit the loop

print(f"You chose {data_form}.")

What form do you want the data to be in? Choose either csv, json, or sql: sql
You chose sql.


In [ ]:
# Handle user-provided data form
if data_form == 'csv':
    # Save DataFrame as CSV
    if (data == 'olympics'):
      print('Data already in csv form')
    elif (data == 'weather'):
      df.to_csv('data.csv', index=False)
      print("Data saved as CSV")
elif data_form == 'json':
    # Save DataFrame as JSON
    if (data == 'weather'):
      print('Data already in json form')
    elif (data == 'olympics'):
      df.to_json('data.json', orient='records', indent=4)
      print("Data saved as JSON.")
elif data_form == 'sql':
    if (data == 'olympics'):
      # Create or connect to an SQLite database
      connection = sqlite3.connect('olympics_data.db')
      cursor = connection.cursor()

      # Save DataFrame to SQL database
      df.to_sql('olympics_data', connection, if_exists='replace', index=False)
      print("Data saved to SQL database.")

      # Commit and close the connection
      connection.commit()
      connection.close()
    if (data == 'weather'):
      # Create or connect to an SQLite database
      connection = sqlite3.connect('weather_data.db')
      cursor = connection.cursor()

      # Save DataFrame to SQL database
      df.to_sql('weather_data', connection, if_exists='replace', index=False)
      print("Data saved to SQL database.")

      # Commit and close the connection
      connection.commit()
      connection.close()

Data saved to SQL database.
